In [ ]:
# default_exp core

# Core

> Core class to login to your DeepWord account and generate synthetic videos

In [ ]:
#exporti
import requests
import json
import urllib3
import base64
import os,cv2
from typing import List, Dict, Union
from pathlib import Path
import pytube
from nbdev.showdoc import *

from dword.utils import URLs

urllib3.disable_warnings()

In [ ]:
#hide
API_KEY = os.environ['API_KEY']
SECRET_KEY = os.environ['SECRET_KEY']

In [ ]:
#export
class DeepWord:
    """
    A class for logging into your DeepWord account in Python and generating videos at scale
    """
    def __init__(self, api_key: str, secret_key: str) -> None:
        """Initialize a new DeepWord object. Login to your DeepWord account to generate api keys.
        """
        self.session = requests.session()
        self.session.verify = False
        self.headers = {'api_key': api_key, 'secerat_key': secret_key, 'Content-Type': 'application/json'}
        self._authenticate_user()

    def _authenticate_user(self):
        url = URLs.validate_token_url
        response = self.session.post(url, headers=self.headers)
        output = self._process_output(response.text)
        if output['success']:
            print('login successful')
        else:
            raise ValueError('Invalid api_key or secret_key')

    @staticmethod
    def _process_output(x): return json.loads(x)

    @property
    def available_credits(self) -> int:
        """Get the number of credits available in your DeepWord account.
        """
        url = URLs.credits_url
        response = self.session.post(url, headers=self.headers)
        try:
            output = self._process_output(response.text)
            return output['available_credits']
        except:
            raise ValueError(response.text)

    def list_videos(self) -> List[Dict]:
        """Get a list of all the videos you've generated using your DeepWord account.
        """
        url = URLs.list_vids_url
        response = self.session.post(url, headers=self.headers)
        try:
            output = self._process_output(response.text)
            return output['data']
        except:
            raise ValueError(response.text)

    def text_to_speech(self, text: str, speaker: str = None, gender: str = None, code: str = None, outfile = 'text2speech.mp3') -> str:
        "Get audio from text using speaker's voice. Output will be stored as outfile"
        # users have to find a simple way to select a speaker like we have on the website with names
        # convert these to set for faster checking
        valid_speakers = ["ar-XA-Wavenet-A","ar-XA-Wavenet-B","ar-XA-Wavenet-C","ar-XA-Standard-A","ar-XA-Standard-B","ar-XA-Standard-C","ar-XA-Standard-D","bn-IN-Standard-A","bn-IN-Standard-B","yue-HK-Standard-A","yue-HK-Standard-B","yue-HK-Standard-C","yue-HK-Standard-D","cs-CZ-Wavenet-A","cs-CZ-Standard-A","da-DK-Wavenet-A","da-DK-Wavenet-C","da-DK-Wavenet-D","da-DK-Wavenet-E","da-DK-Standard-A","da-DK-Standard-C","da-DK-Standard-D","da-DK-Standard-E","nl-NL-Wavenet-A","nl-NL-Wavenet-B","nl-NL-Wavenet-C","nl-NL-Wavenet-D","nl-NL-Wavenet-E","nl-NL-Standard-A","nl-NL-Standard-B","nl-NL-Standard-C","nl-NL-Standard-D","nl-NL-Standard-E","en-AU-Wavenet-A","en-AU-Wavenet-B","en-AU-Wavenet-C","en-AU-Wavenet-D","en-AU-Standard-A","en-AU-Standard-B","en-AU-Standard-C","en-AU-Standard-D","en-IN-Wavenet-A","en-IN-Wavenet-B","en-IN-Wavenet-C","en-IN-Wavenet-D","en-IN-Standard-A","en-IN-Standard-B","en-IN-Standard-C","en-IN-Standard-D","en-GB-Wavenet-A","en-GB-Wavenet-B","en-GB-Wavenet-C","en-GB-Wavenet-D","en-GB-Wavenet-F","en-GB-Standard-A","en-GB-Standard-B","en-GB-Standard-C","en-GB-Standard-D","en-GB-Standard-F","en-US-Wavenet-A","en-US-Wavenet-B","en-US-Wavenet-C","en-US-Wavenet-D","en-US-Wavenet-E","en-US-Wavenet-F","en-US-Wavenet-G","en-US-Wavenet-H","en-US-Wavenet-I","en-US-Wavenet-J","en-US-Standard-B","en-US-Standard-C","en-US-Standard-D","en-US-Standard-E","en-US-Standard-G","en-US-Standard-H","en-US-Standard-I","en-US-Standard-J","fil-PH-Wavenet-A","fil-PH-Wavenet-B","fil-PH-Wavenet-C","fil-PH-Wavenet-D","fil-PH-Standard-A","fil-PH-Standard-B","fil-PH-Standard-C","fil-PH-Standard-D","fi-FI-Wavenet-A","fi-FI-Standard-A","fr-CA-Wavenet-A","fr-CA-Wavenet-B","fr-CA-Wavenet-C","fr-CA-Wavenet-D","fr-CA-Standard-A","fr-CA-Standard-B","fr-CA-Standard-C","fr-CA-Standard-D","fr-FR-Wavenet-A","fr-FR-Wavenet-B","fr-FR-Wavenet-C","fr-FR-Wavenet-D","fr-FR-Wavenet-E","fr-FR-Standard-A","fr-FR-Standard-B","fr-FR-Standard-C","fr-FR-Standard-D","fr-FR-Standard-E","de-DE-Wavenet-A","de-DE-Wavenet-B","de-DE-Wavenet-C","de-DE-Wavenet-D","de-DE-Wavenet-E","de-DE-Wavenet-F","de-DE-Standard-A","de-DE-Standard-B","de-DE-Standard-E","de-DE-Standard-F","el-GR-Wavenet-A","el-GR-Standard-A","gu-IN-Standard-A","gu-IN-Standard-B","hi-IN-Wavenet-A","hi-IN-Wavenet-B","hi-IN-Wavenet-C","hi-IN-Wavenet-D","hi-IN-Standard-A","hi-IN-Standard-B","hi-IN-Standard-C","hi-IN-Standard-D","hu-HU-Wavenet-A","hu-HU-Standard-A","id-ID-Wavenet-A","id-ID-Wavenet-B","id-ID-Wavenet-C","id-ID-Wavenet-D","id-ID-Standard-A","id-ID-Standard-B","id-ID-Standard-C","id-ID-Standard-D","it-IT-Wavenet-A","it-IT-Wavenet-B","it-IT-Wavenet-C","it-IT-Wavenet-D","it-IT-Standard-A","it-IT-Standard-B","it-IT-Standard-C","it-IT-Standard-D","ja-JP-Wavenet-A","ja-JP-Wavenet-B","ja-JP-Wavenet-C","ja-JP-Wavenet-D","ja-JP-Standard-A","ja-JP-Standard-B","ja-JP-Standard-C","ja-JP-Standard-D","kn-IN-Standard-A","kn-IN-Standard-B","ko-KR-Wavenet-A","ko-KR-Wavenet-B","ko-KR-Wavenet-C","ko-KR-Wavenet-D","ko-KR-Standard-A","ko-KR-Standard-B","ko-KR-Standard-C","ko-KR-Standard-D","ml-IN-Standard-A","ml-IN-Standard-B","cmn-CN-Wavenet-A","cmn-CN-Wavenet-B","cmn-CN-Wavenet-C","cmn-CN-Wavenet-D","cmn-TW-Wavenet-A","cmn-TW-Wavenet-B","cmn-TW-Wavenet-C","cmn-CN-Standard-A","cmn-CN-Standard-B","cmn-CN-Standard-C","cmn-CN-Standard-D","cmn-TW-Standard-A","cmn-TW-Standard-B","cmn-TW-Standard-C","nb-NO-Wavenet-A","nb-NO-Wavenet-B","nb-no-Wavenet-E","nb-NO-Wavenet-C","nb-NO-Wavenet-D","nb-NO-Standard-A","nb-NO-Standard-B","nb-NO-Standard-C","nb-NO-Standard-D","nb-no-Standard-E","pl-PL-Wavenet-A","pl-PL-Wavenet-B","pl-PL-Wavenet-C","pl-PL-Wavenet-D","pl-PL-Wavenet-E","pl-PL-Standard-A","pl-PL-Standard-B","pl-PL-Standard-C","pl-PL-Standard-D","pl-PL-Standard-E","pt-BR-Wavenet-A","pt-BR-Standard-A","pt-PT-Wavenet-A","pt-PT-Wavenet-B","pt-PT-Wavenet-C","pt-PT-Wavenet-D","pt-PT-Standard-A","pt-PT-Standard-B","pt-PT-Standard-C","pt-PT-Standard-D","ru-RU-Wavenet-A","ru-RU-Wavenet-B","ru-RU-Wavenet-C","ru-RU-Wavenet-D","ru-RU-Wavenet-E","ru-RU-Standard-A","ru-RU-Standard-B","ru-RU-Standard-C","ru-RU-Standard-D","ru-RU-Standard-E","sk-SK-Wavenet-A","sk-SK-Standard-A","es-ES-Wavenet-B","es-ES-Standard-A","es-ES-Standard-B","sv-SE-Wavenet-A","sv-SE-Standard-A","ta-IN-Standard-A","ta-IN-Standard-B","te-IN-Standard-A","te-IN-Standard-B","th-TH-Standard-A","tr-TR-Wavenet-A","tr-TR-Wavenet-B","tr-TR-Wavenet-C","tr-TR-Wavenet-D","tr-TR-Wavenet-E","tr-TR-Standard-A","tr-TR-Standard-B","tr-TR-Standard-C","tr-TR-Standard-D","tr-TR-Standard-E","uk-UA-Wavenet-A","uk-UA-Standard-A","vi-VN-Wavenet-A","vi-VN-Wavenet-B","vi-VN-Wavenet-C","vi-VN-Wavenet-D","vi-VN-Standard-A","vi-VN-Standard-B","vi-VN-Standard-C","vi-VN-Standard-D"]
        # change this parameter to language, and have a dict mapping between what they select and the code
        valid_codes = ["ar-XA", "bn-IN", "yue-HK", "cs-CZ", "da-DK", "nl-NL", "en-AU", "en-IN", "en-GB", "en-US", "fil-PH", "fi-FI", "fr-CA", "fr-FR", "de-DE", "el-GR", "gu-IN", "hi-IN", "hu-HU", "id-ID", "it-IT", "ja-JP", "kn-IN", "ko-KR", "ml-IN", "cmn-CN", "cmn-TW", "nb-NO", "pl-PL", "pt-BR", "pt-PT", "ru-RU", "sk-SK", "es-ES", "sv-SE", "ta-IN", "te-IN", "th-TH", "tr-TR", "uk-UA", "vi-VN"]
        valid_genders = ['MALE','FEMALE']

        if not speaker: speaker = random.choice(valid_speakers)
        if not code: code = random.choice(valid_codes)
        if not gender: gender = random.choice(valid_genders)

        if speaker not in valid_speakers: raise ValueError(f'Invalid speaker. Got {speaker}, expected one of {valid_speakers}')
        if code not in valid_codes: raise ValueError(f'Invalid code. Got {code}, expected one of {valid_codes}')
        if gender not in valid_genders: raise ValueError(f'Invalid code. Got {gender}, expected one of {valid_genders}')

        payload='{"text":"%s","name":"%s","gender":"%s","code":"%s"}'% (text,speaker,gender,code)
        url = URLs.txt2speech_url
        response = self.session.post(url, headers=self.headers,data=payload)

        try:
            decode_bytes = base64.b64decode(response.text)
            with open(outfile, "wb") as wav_file:
                wav_file.write(decode_bytes)
            return (f"Successfully generated audio file {outfile}")
        except Exception as e:
            raise ValueError(response.text)

    def download_video(self, video_id: str) -> None:
        """Download one of the synthetically generated videos on your DeepWord account. 
           The video id can be found using the ``list_generated_videos()`` function. The video
           should have finished processing to be downloadable.
           Optionally, you can use download_all_videos().
        """
        url = URLs.download_vid_url + video_id
        response = self.session.get(url, headers=self.headers)
        if response.json()['status'] is False:
            raise ValueError("Video is still processing. Unable to download it at this time.")
        try:
            r = requests.get(response.json()['video_url'], stream=True)
            with open(response.json()['video_name'], 'wb') as f:
                for chunk in r.iter_content(chunk_size=1024): 
                    if chunk: # filter out keep-alive new chunks
                        f.write(chunk)
            return (f"Successfully downloaded video {video_id}!")
        except Exception as e:
            raise ValueError(response.text)

    def download_all_videos(self, folder: Union[str, Path]  = 'downloaded_videos') -> None:
        """Download all vidoes generated with your DeepWord account. You can also pass
        a folder or nested folders where you want the vidoes to be saved.
        """
        url = URLs.list_vids_url
        path = Path().cwd()
        
        folder = Path(folder)
        folder.mkdir(parents = True, exist_ok = True)

        response = self.session.post(url, headers=self.headers)
        try:
            for item in response.json()['data']:
                r = requests.get(item['video_url'], stream=True)

                fname1 = item['title'].replace(".mp4",'')+'.mp4'
                full_path = f'{path/folder/fname1}'

                with open(full_path, 'wb') as f:
                    for chunk in r.iter_content(chunk_size=1024): 
                        if chunk: # filter out keep-alive new chunks
                            f.write(chunk)
            print(f"Successfully downloaded all videos in folder {folder}!")
        except Exception as e:
            raise ValueError(response.text)

    def download_youtube_video(self, url: str, types: str = 'video', start_time: int = None, end_time: int = None):
        """Download a video from YouTube. You can also donwload an audio and provide start and
           end times to download a trimmed version.
        """
        if not start_time and not end_time:
            # there should be an if else inside this
            # if types == 'audio' then it should download the full mp3 else mp4
            pytube.YouTube(url).streams.get_highest_resolution().download()
            return ("downloaded youtube video successfully!")

        if start_time > end_time:
            raise ValueError('Start time cannot be greater than end time')
        
        if (start_time is None and end_time) or (end_time is None and start_time is not None):
            raise ValueError('Both start and end times should be provided')

        start_time = to_hhmmss(int(start_time))
        end_time = to_hhmmss(int(end_time))
        payload = '{"end_time": "%s","start_time": "%s","type": "%s","url": "%s"}' % (end_time,start_time,types,url)
        url = URLs.download_yt_vid_url
        response = self.session.post(url, headers=self.headers, data=payload)
        try:
            r = self.session.get(response.json()['video_url'], stream=True,verify=False)
            fname = response.json()['video_url'].split("/")[-1]
            with open(fname, 'wb') as f:
                for chunk in r.iter_content(chunk_size=1024): 
                    if chunk: # filter out keep-alive new chunks
                        f.write(chunk)
            print(f"Successfully downloaded the YouTube video {fname}")
            return fname
        except Exception as e:
            raise ValueError(response.text)
    
    def download_audio_samples(self):
        """Download all the audio samples available on the DeepWord website
        """
        folder = Path().cwd() / 'audio_samples'
        folder.mkdir(exist_ok = True)

        url = URLs.api_get_audio_sample
        response = self.session.post(url, headers=self.headers)
        try:
            for dic in self._process_output(response.text)['sample_audio_files']:
                doc = self.session.get(dic['audio_url'])
                fname = folder / (dic['title']+dic['extension'])
                f = open(fname,"wb")
                f.write(doc.content)
                f.close()
            return ("Successfully downloaded all audio samples")
        except Exception as e:
            raise ValueError(response.text)

    def download_video_actors(self):
        """Download all the video actors available on the DeepWord website.
        """
        folder = Path().cwd() / 'video_actors'
        folder.mkdir(exist_ok = True)

        url = URLs.api_get_video_actors
        response = self.session.post(url, headers=self.headers)
        try:
            for dic in self._process_output(response.text)['sample_video_files']:
                with self.session.get(dic['video_url'], stream=True) as r:
                    r.raise_for_status()
                    fname = folder / (dic['title']+dic['extension'])
                    with open(fname, 'wb') as f:
                        for chunk in r.iter_content(chunk_size=8192): 
                            f.write(chunk)
            return ("Successfully downloaded all video actors")
        except Exception as e:
            raise ValueError(response.text) 

    def generate_video(self, video: str, audio: str, title: str = None):
        """Generate a synthetic video using a video of a person talking and the audio
           you want them to say. You can check the status of the video using
           ``list_generated_videos`` and download it using ``download_video`` or
           ``download_all_videos``
        """
        if not _exists(video): raise ValueError(f'File not found {video}')
        if not _exists(audio): raise ValueError(f'File not found {audio}')
        payload = {}
        if title is not None:
            payload={'name': title}
        self.headers.pop("Content-Type", None)
        url = URLs.generate_vid_url
        files = {'video_file': open(video,'rb'),'audio_file': open(audio,'rb')}
        response = self.session.post(url, headers=self.headers,files=files,data=payload)
        try:
            print('Generating video. This will take a few minutes.')
            return response.json()
        except Exception as e:
            raise ValueError(e)

In [ ]:
account = DeepWord(API_KEY, SECRET_KEY)

login successful


## Account details

In [ ]:
show_doc(DeepWord.available_credits)

<h4 id="DeepWord.available_credits" class="doc_header"><code>DeepWord.available_credits</code><a href="" class="source_link" style="float:right">[source]</a></h4>

Get the number of credits available in your DeepWord account.
        

In [ ]:
account.available_credits

10014

In [ ]:
show_doc(DeepWord.list_videos)

<h4 id="DeepWord.list_videos" class="doc_header"><code>DeepWord.list_videos</code><a href="__main__.py#L38" class="source_link" style="float:right">[source]</a></h4>

> <code>DeepWord.list_videos</code>()

Get a list of all the videos you've generated using your DeepWord account.
        

In [ ]:
all_videos = account.list_videos()
all_videos[0]

{'email': 'blablabla@yopmail.com',
 'thumbnail': 'thubnail-u5mfeqpkn6uwhsn.png',
 'title': 'Noelle',
 'video_url': 'https://videos-deep-word123.s3.us-east-2.amazonaws.com/output_data/u5mfeqpkn6uwhsn.mp4',
 'video_duration': '53.952',
 'video_id': 'u5mfeqpkn6uwhsn',
 'generate_date': '2021-04-07T02:52:57.000Z',
 'output_status': 'Progress'}

## Downloading synthetically generated videos

In [ ]:
show_doc(DeepWord.download_video)

<h4 id="DeepWord.download_video" class="doc_header"><code>DeepWord.download_video</code><a href="__main__.py#L78" class="source_link" style="float:right">[source]</a></h4>

> <code>DeepWord.download_video</code>(**`video_id`**:`str`)

Download one of the synthetically generated videos on your DeepWord account. 
The video id can be found using the ``list_generated_videos()`` function. The video
should have finished processing to be downloadable.
Optionally, you can use download_all_videos().

In [ ]:
show_doc(DeepWord.download_all_videos)

<h4 id="DeepWord.download_all_videos" class="doc_header"><code>DeepWord.download_all_videos</code><a href="__main__.py#L98" class="source_link" style="float:right">[source]</a></h4>

> <code>DeepWord.download_all_videos</code>(**`folder`**:`Union`\[`str`, `Path`\]=*`'downloaded_videos'`*)

Download all vidoes generated with your DeepWord account. You can also pass
a folder or nested folders where you want the vidoes to be saved.

In [ ]:
account.download_all_videos('my_videos')

Successfully downloaded all videos in folder my_videos!


## Downloading sample data

In [ ]:
show_doc(DeepWord.download_video_actors)

<h4 id="DeepWord.download_video_actors" class="doc_header"><code>DeepWord.download_video_actors</code><a href="__main__.py#L176" class="source_link" style="float:right">[source]</a></h4>

> <code>DeepWord.download_video_actors</code>()

Download all the video actors available on the DeepWord website.
        

In [ ]:
account.download_video_actors()

'Successfully downloaded all video actors'

In [ ]:
!ls video_actors/eo_actors

ls: video_actors/eo_actors: No such file or directory


In [ ]:
show_doc(DeepWord.download_audio_samples)

<h4 id="DeepWord.download_audio_samples" class="doc_header"><code>DeepWord.download_audio_samples</code><a href="__main__.py#L157" class="source_link" style="float:right">[source]</a></h4>

> <code>DeepWord.download_audio_samples</code>()

Download all the audio samples available on the DeepWord website
        

In [ ]:
account.download_audio_samples()

'Successfully downloaded all audio samples'

In [ ]:
!ls audio_samples

Amy Poehler.mp3            Mark Cuban.mp3
Boeing CEO.mp3             Martin Scorsese.mp3
Chamath Phalihapitiya.mp3  Michael Scott.mp3
Chinese Instructor.mp3     Neil deGrasse Tyson.mp3
Edward Snowden.mp3         Peyton Manning.mp3
Elizabeth Holmes.mp3       Scottie Pippen.mp3
Financial Advice.mp3       Shaq.mp3
Footbal Meme.mp3           Swim Coach.mp3
George Hotz.mp3            Tampa Bay Rays Manager.mp3
Kevin Hart.mp3             [Korean] Bong Joon Ho.mp3


## Downloading YouTube videos

In [ ]:
show_doc(DeepWord.download_youtube_video)

<h4 id="DeepWord.download_youtube_video" class="doc_header"><code>DeepWord.download_youtube_video</code><a href="__main__.py#L124" class="source_link" style="float:right">[source]</a></h4>

> <code>DeepWord.download_youtube_video</code>(**`url`**:`str`, **`types`**:`str`=*`'video'`*, **`start_time`**:`int`=*`None`*, **`end_time`**:`int`=*`None`*)

Download a video from YouTube. You can also donwload an audio and provide start and
end times to download a trimmed version.

In [ ]:
url = 'https://youtu.be/_81yOz-ujug'

In [ ]:
account.download_youtube_video(url)

'downloaded youtube video successfully!'

## Creating synthetic videos

In [ ]:
show_doc(DeepWord.generate_video)

<h4 id="DeepWord.generate_video" class="doc_header"><code>DeepWord.generate_video</code><a href="__main__.py#L196" class="source_link" style="float:right">[source]</a></h4>

> <code>DeepWord.generate_video</code>(**`video`**:`str`, **`audio`**:`str`, **`title`**:`str`=*`None`*)

Generate a synthetic video using a video of a person talking and the audio
you want them to say. You can check the status of the video using
``list_generated_videos`` and download it using ``download_video`` or
``download_all_videos``

```python
account.generate_video('video_actors/Anna.mp4', 'audio_samples/Amy Poehler.mp3')
```

In [ ]:
#hide
from nbdev.export import notebook2script
notebook2script()

Converted 00_core.ipynb.
Converted 01_utils.ipynb.
Converted index.ipynb.
